# Training MNet

In [19]:
import numpy as np
from keras import backend as K
import scipy
from skimage.measure import label, regionprops
import os
from keras.preprocessing import image
import tensorflow as tf
from keras.optimizers import SGD
from keras.layers import (
    Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, BatchNormalization,
    Activation, Conv2DTranspose, Multiply, Add, AveragePooling2D, Average
)
from keras.models import Model
import random

## Utils

In [20]:
# Define helper functions
def mk_dir(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
    return dir_path

def return_list(data_path, data_type):
    file_list = [file for file in os.listdir(data_path) if file.lower().endswith(data_type)]
    print(str(len(file_list)))
    return file_list

def train_loader(data_list, data_path, mask_path, input_size):
    for lineIdx in range(len(data_list)):
        temp_txt = data_list[lineIdx]

        train_img = np.asarray(image.load_img(
            os.path.join(data_path, temp_txt),
            target_size=(input_size, input_size, 3))
        ).astype('float32')

        img_mask = np.asarray(image.load_img(
            os.path.join(mask_path, temp_txt),
            target_size=(input_size, input_size, 3))) / 255.0

        img_mask = img_mask.astype('float32')

        yield (train_img, (img_mask, img_mask, img_mask, img_mask, img_mask))


def tf_data_loader(data_list, data_path, mask_path, input_size):
    dataset = tf.data.Dataset.from_generator(
        lambda: train_loader(data_list, data_path, mask_path, input_size),
        output_signature=(
            tf.TensorSpec(shape=(input_size, input_size, 3), dtype=tf.float32),
            tuple([tf.TensorSpec(shape=(input_size, input_size, 3), dtype=tf.float32)] * 5)
        )
    )
    return dataset

In [21]:
# Training Settings
result_path = mk_dir('/kaggle/working/output_deep_model/')
pre_model_file = '/kaggle/input/deep-model/deep_model/Model_MNet_REFUGE.h5'
save_model_file = result_path + 'Model_MNet_REFUGE_v2.h5'

root_path = '/kaggle/input/training-data-polar/training_crop/'
train_data_path = root_path + 'data/'
train_mask_path = root_path + 'label/'

val_data_path = root_path + 'val_data/data/'
val_mask_path = root_path + 'val_data/label/'

train_list = return_list(train_data_path, '.png')
val_list = return_list(val_data_path, '.png')

1400
300


## Model MNet

In [22]:
from keras.layers import (
    Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, BatchNormalization,
    Activation, Conv2DTranspose, Multiply, Add, AveragePooling2D, Average
)
from keras.models import Model

def attention_gate(x, g, inter_channels):
    # x: Input feature map from the encoder (skip connection)
    # g: Gating signal from the decoder
    # inter_channels: Number of filters for intermediate computations

    theta_x = Conv2D(inter_channels, (2, 2), strides=(2, 2), padding='same')(x)
    phi_g = Conv2D(inter_channels, (1, 1), padding='same')(g)
    add = Add()([theta_x, phi_g])
    relu = Activation('relu')(add)
    psi = Conv2D(1, (1, 1), padding='same')(relu)
    sigmoid = Activation('sigmoid')(psi)
    upsampled = UpSampling2D(size=(2, 2), interpolation='bilinear')(sigmoid)
    attn_coefficients = Multiply()([x, upsampled])
    return attn_coefficients

def DeepModel(size_set=800):

    img_input = Input(shape=(size_set, size_set, 3))

    # Scaled inputs
    scale_img_2 = AveragePooling2D(pool_size=(2, 2))(img_input)
    scale_img_3 = AveragePooling2D(pool_size=(2, 2))(scale_img_2)
    scale_img_4 = AveragePooling2D(pool_size=(2, 2))(scale_img_3)

    # Block 1
    conv1 = Conv2D(32, (3, 3), padding='same', name='block1_conv1')(img_input)
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation('relu')(conv1)
    conv1 = Conv2D(32, (3, 3), padding='same', name='block1_conv2')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation('relu')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    # Block 2
    input2 = Conv2D(64, (3, 3), padding='same', name='block2_input1')(scale_img_2)
    input2 = BatchNormalization()(input2)
    input2 = Activation('relu')(input2)
    input2 = concatenate([input2, pool1], axis=3)
    conv2 = Conv2D(64, (3, 3), padding='same', name='block2_conv1')(input2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation('relu')(conv2)
    conv2 = Conv2D(64, (3, 3), padding='same', name='block2_conv2')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation('relu')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    # Block 3
    input3 = Conv2D(128, (3, 3), padding='same', name='block3_input1')(scale_img_3)
    input3 = BatchNormalization()(input3)
    input3 = Activation('relu')(input3)
    input3 = concatenate([input3, pool2], axis=3)
    conv3 = Conv2D(128, (3, 3), padding='same', name='block3_conv1')(input3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('relu')(conv3)
    conv3 = Conv2D(128, (3, 3), padding='same', name='block3_conv2')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('relu')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    # Block 4
    input4 = Conv2D(256, (3, 3), padding='same', name='block4_input1')(scale_img_4)
    input4 = BatchNormalization()(input4)
    input4 = Activation('relu')(input4)
    input4 = concatenate([input4, pool3], axis=3)
    conv4 = Conv2D(256, (3, 3), padding='same', name='block4_conv1')(input4)
    conv4 = BatchNormalization()(conv4)
    conv4 = Activation('relu')(conv4)
    conv4 = Conv2D(256, (3, 3), padding='same', name='block4_conv2')(conv4)
    conv4 = BatchNormalization()(conv4)
    conv4 = Activation('relu')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    # Bridge
    conv5 = Conv2D(512, (3, 3), padding='same', name='block5_conv1')(pool4)
    conv5 = BatchNormalization()(conv5)
    conv5 = Activation('relu')(conv5)
    conv5 = Conv2D(512, (3, 3), padding='same', name='block5_conv2')(conv5)
    conv5 = BatchNormalization()(conv5)
    conv5 = Activation('relu')(conv5)

    # Decoder with Attention Gates
    # Up Block 6
    attn4 = attention_gate(conv4, conv5, inter_channels=256)
    up6 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same', name='block6_dconv')(conv5)
    up6 = concatenate([up6, attn4], axis=3)
    conv6 = Conv2D(256, (3, 3), padding='same', name='block6_conv1')(up6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Activation('relu')(conv6)
    conv6 = Conv2D(256, (3, 3), padding='same', name='block6_conv2')(conv6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Activation('relu')(conv6)

    # Up Block 7
    attn3 = attention_gate(conv3, conv6, inter_channels=128)
    up7 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same', name='block7_dconv')(conv6)
    up7 = concatenate([up7, attn3], axis=3)
    conv7 = Conv2D(128, (3, 3), padding='same', name='block7_conv1')(up7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Activation('relu')(conv7)
    conv7 = Conv2D(128, (3, 3), padding='same', name='block7_conv2')(conv7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Activation('relu')(conv7)

    # Up Block 8
    attn2 = attention_gate(conv2, conv7, inter_channels=64)
    up8 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same', name='block8_dconv')(conv7)
    up8 = concatenate([up8, attn2], axis=3)
    conv8 = Conv2D(64, (3, 3), padding='same', name='block8_conv1')(up8)
    conv8 = BatchNormalization()(conv8)
    conv8 = Activation('relu')(conv8)
    conv8 = Conv2D(64, (3, 3), padding='same', name='block8_conv2')(conv8)
    conv8 = BatchNormalization()(conv8)
    conv8 = Activation('relu')(conv8)

    # Up Block 9
    attn1 = attention_gate(conv1, conv8, inter_channels=32)
    up9 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same', name='block9_dconv')(conv8)
    up9 = concatenate([up9, attn1], axis=3)
    conv9 = Conv2D(32, (3, 3), padding='same', name='block9_conv1')(up9)
    conv9 = BatchNormalization()(conv9)
    conv9 = Activation('relu')(conv9)
    conv9 = Conv2D(32, (3, 3), padding='same', name='block9_conv2')(conv9)
    conv9 = BatchNormalization()(conv9)
    conv9 = Activation('relu')(conv9)

    # Side Outputs with 1 channel each
    side6 = UpSampling2D(size=(8, 8))(conv6)
    side7 = UpSampling2D(size=(4, 4))(conv7)
    side8 = UpSampling2D(size=(2, 2))(conv8)
    out6 = Conv2D(3, (1, 1), activation='softmax', name='side_63')(side6)
    out7 = Conv2D(3, (1, 1), activation='softmax', name='side_73')(side7)
    out8 = Conv2D(3, (1, 1), activation='softmax', name='side_83')(side8)
    out9 = Conv2D(3, (1, 1), activation='softmax', name='side_93')(conv9)
    
    # Final Output with name 'final_output'
    out10 = Average(name='final_output')([out6, out7, out8, out9])

    model = Model(inputs=[img_input], outputs=[out6, out7, out8, out9, out10])

    return model

# Training 

In [23]:
# # Set the root and data paths
# root_path = '/kaggle/input/training-data-polar/training_crop/'
# train_data_path = os.path.join(root_path, 'data/')
# train_mask_path = os.path.join(root_path, 'label/')

# val_data_path = os.path.join(root_path, 'val_data/data/')
# val_mask_path = os.path.join(root_path, 'val_data/label/')

# # Define image dimensions
# IMAGE_SIZE = 800  # As per your model's input size

# # Function to load and preprocess images
# def load_image(path, target_size):
#     img = load_img(path, target_size=(target_size, target_size))
#     img = img_to_array(img)
#     img = img / 255.0  # Normalize to [0, 1]
#     return img

# # Function to load and preprocess masks
# def load_mask(path, target_size):
#     mask = load_img(path, target_size=(target_size, target_size), color_mode='grayscale')
#     mask = img_to_array(mask)
#     mask = mask / 255.0  # Normalize to [0, 1]
#     mask = np.round(mask)  # Ensure binary masks
#     return mask

# # Custom data generator
# class DataGenerator(Sequence):
#     def __init__(self, image_filenames, mask_filenames, image_dir, mask_dir, batch_size, target_size, shuffle=True):
#         self.image_filenames = image_filenames
#         self.mask_filenames = mask_filenames
#         self.image_dir = image_dir
#         self.mask_dir = mask_dir
#         self.batch_size = batch_size
#         self.target_size = target_size
#         self.shuffle = shuffle
#         self.on_epoch_end()
    
#     def __len__(self):
#         return int(np.ceil(len(self.image_filenames) / self.batch_size))
    
#     def __getitem__(self, index):
#         # Generate indexes of the batch
#         batch_indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        
#         # Find list of IDs
#         batch_image_filenames = [self.image_filenames[k] for k in batch_indexes]
#         batch_mask_filenames = [self.mask_filenames[k] for k in batch_indexes]
        
#         # Generate data
#         X, Y = self.__data_generation(batch_image_filenames, batch_mask_filenames)
        
#         return X, Y
    
#     def on_epoch_end(self):
#         self.indexes = np.arange(len(self.image_filenames))
#         if self.shuffle:
#             np.random.shuffle(self.indexes)
    
#     def __data_generation(self, batch_image_filenames, batch_mask_filenames):
#         X = np.empty((len(batch_image_filenames), self.target_size, self.target_size, 3), dtype=np.float32)
#         Y = np.empty((len(batch_mask_filenames), self.target_size, self.target_size, 1), dtype=np.float32)
        
#         for i, (img_name, mask_name) in enumerate(zip(batch_image_filenames, batch_mask_filenames)):
#             img_path = os.path.join(self.image_dir, img_name)
#             mask_path = os.path.join(self.mask_dir, mask_name)
            
#             # Load and preprocess images and masks
#             img = load_image(img_path, self.target_size)
#             mask = load_mask(mask_path, self.target_size)
            
#             X[i] = img
#             Y[i] = mask
        
#         # Prepare outputs as a dictionary mapping output names to targets
#         Y_multi = {
#             'side_63': Y,
#             'side_73': Y,
#             'side_83': Y,
#             'side_93': Y,
#             'final_output': Y
#         }
#         return X, Y_multi


# # Get the list of image and mask filenames for training
# train_image_files = sorted(os.listdir(train_data_path))
# train_mask_files = sorted(os.listdir(train_mask_path))

# # Get the list of image and mask filenames for validation
# val_image_files = sorted(os.listdir(val_data_path))
# val_mask_files = sorted(os.listdir(val_mask_path))

# # Create data generators
# batch_size = 4  # Adjust based on your GPU memory

# train_generator = DataGenerator(
#     train_image_files, train_mask_files,
#     train_data_path, train_mask_path,
#     batch_size=batch_size, target_size=IMAGE_SIZE, shuffle=True
# )

# val_generator = DataGenerator(
#     val_image_files, val_mask_files,
#     val_data_path, val_mask_path,
#     batch_size=batch_size, target_size=IMAGE_SIZE, shuffle=False
# )

# # Define the model
# model = DeepModel(size_set=IMAGE_SIZE)

# # Compile the model with dictionaries for losses and loss_weights
# losses = {
#     'side_63': 'binary_crossentropy',
#     'side_73': 'binary_crossentropy',
#     'side_83': 'binary_crossentropy',
#     'side_93': 'binary_crossentropy',
#     'final_output': 'binary_crossentropy'
# }

# loss_weights = {
#     'side_63': 0.1,
#     'side_73': 0.1,
#     'side_83': 0.1,
#     'side_93': 0.1,
#     'final_output': 0.6
# }

# def dice_coefficient(y_true, y_pred):
#     smooth = 1.  # Avoid division by zero
#     y_true_f = K.flatten(y_true)
#     y_pred_f = K.flatten(y_pred)
#     intersection = K.sum(y_true_f * y_pred_f)
#     return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

# metrics = {
#     'side_63': ['accuracy'],
#     'side_73': ['accuracy'],
#     'side_83': ['accuracy'],
#     'side_93': ['accuracy'],
#     'final_output': ['accuracy']#, dice_coefficie
# }

# model.compile(
#     optimizer=Adam(learning_rate=1e-4),
#     loss=losses,
#     loss_weights=loss_weights,
#     metrics= metrics,
# )

# # Callbacks
# checkpoint = ModelCheckpoint(
#     '/kaggle/working/best_model.weights.h5',
#     monitor='val_loss',
#     verbose=1,
#     save_best_only=True,
#     mode='min',
#     save_weights_only=True
# )

# reduce_lr = ReduceLROnPlateau(
#     monitor='val_loss',
#     factor=0.5,
#     patience=5,
#     min_lr=1e-7,
#     verbose=1
# )

# early_stopping = EarlyStopping(
#     monitor='val_loss',
#     patience=10,
#     restore_best_weights=True
# )

# callbacks_list = [checkpoint, reduce_lr, early_stopping]

# # Train the model
# epochs = 50  # Adjust as needed
# print("Start training")
# model.fit(
#     train_generator,
#     validation_data=val_generator,
#     epochs=epochs,
#     callbacks=callbacks_list
# )

In [ ]:
Total_iter = 10
nb_epoch_setting = 3
input_size = 400
optimizer_setting = SGD(learning_rate=0.0001, momentum=0.9)

# Build and compile the model
my_model = DeepModel(size_set=input_size)
# my_model.load_weights(pre_model_file, by_name=True)

my_model.compile(optimizer=optimizer_setting, loss='categorical_crossentropy',
                 loss_weights=[0.1, 0.1, 0.1, 0.1, 0.6])

print(my_model.input_shape)  # Should output (None, 400, 400, 3)

# Data Preparation
train_dataset = tf_data_loader(train_list, train_data_path, train_mask_path, input_size)
val_dataset = tf_data_loader(val_list, val_data_path, val_mask_path, input_size)

train_dataset = train_dataset.shuffle(buffer_size=len(train_list)).batch(8).repeat().prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(8).repeat().prefetch(tf.data.AUTOTUNE)

batch_size = 8
steps_per_epoch = len(train_list) // batch_size
validation_steps = len(val_list) // batch_size

# Verify data shapes
for images, labels in train_dataset.take(1):
    print("Image shape:", images.shape)  # Expected: (batch_size, input_size, input_size, 3)
    for idx, label in enumerate(labels):
        print(f"Label {idx+1} shape:", label.shape)  # Expected: (batch_size, input_size, input_size, 3)

# Training Loop
loss_max = 10000
for idx_iter in range(Total_iter):
    print("Training started")
    model_return = my_model.fit(
        x=train_dataset,
        validation_data=val_dataset,
        epochs=nb_epoch_setting,
        steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps,
        verbose=1
    )
    val_loss = model_return.history['val_loss'][0]
    train_loss = model_return.history['loss'][0]
    if val_loss < loss_max:
        my_model.save(save_model_file)
        loss_max = val_loss
        print('[Save] training iter: ' + str(idx_iter+1) +
              ', train_loss: ' + str(train_loss) +
              ', val_loss: ' + str(val_loss))
    else:
        print('[None] training iter: ' + str(idx_iter+1) +
              ', train_loss: ' + str(train_loss) +
              ', val_loss: ' + str(val_loss))

(None, 400, 400, 3)
Image shape: (8, 400, 400, 3)
Label 1 shape: (8, 400, 400, 3)
Label 2 shape: (8, 400, 400, 3)
Label 3 shape: (8, 400, 400, 3)
Label 4 shape: (8, 400, 400, 3)
Label 5 shape: (8, 400, 400, 3)
Training started
Epoch 1/3


I0000 00:00:1733285619.664391      96 service.cc:145] XLA service 0x7a9d2400a3f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733285619.664449      96 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1733285655.511448      96 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'input_add_reduce_fusion_13', 972 bytes spill stores, 904 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_compare_reduce_fusion', 96 bytes spill stores, 96 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_add_reduce_fusion_3', 104 bytes spill stores, 104 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_reduce_select_fusion', 64 bytes spill stores, 64 bytes spill loads

I0000 00:00:1733285655.541913      96 device_compiler.h:188] Compiled cluster u

175/175 ━━━━━━━━━━━━━━━━━━━━ 153s 428ms/step - loss: 0.8556 - val_loss: 0.7268
Epoch 2/3


## Validation try

In [ ]:
import matplotlib.pyplot as plt

# Function to load a single image and mask
def load_single_image(img_path, mask_path, target_size):
    img = load_image(img_path, target_size)
    mask = load_mask(mask_path, target_size)
    return img, mask

# Select a sample from the validation set
sample_index = 0
sample_img_name = val_image_files[sample_index]
sample_mask_name = val_mask_files[sample_index]

sample_img_path = os.path.join(val_data_path, sample_img_name)
sample_mask_path = os.path.join(val_mask_path, sample_mask_name)

# Load the image and mask
img, mask = load_single_image(sample_img_path, sample_mask_path, IMAGE_SIZE)
img_input = np.expand_dims(img, axis=0)  # Add batch dimension

# Predict
predictions = model.predict(img_input)
final_output = predictions[-1][0]  # Extract the first sample from the batch

# Threshold the output to get binary mask
binary_mask = (final_output > 0.5).astype(np.uint8)

# Visualization
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(img)
plt.title('Input Image')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(mask.squeeze())
plt.title('Ground Truth Mask')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(binary_mask.squeeze())
plt.title('Predicted Mask')
plt.axis('off')

plt.show()